In [22]:
import os, ssl, sys

if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
sys.path.append('../code/')

import numpy as np
import tensorflow as tf
import gpflow

from pathlib import Path
from gpflow.likelihoods import Gaussian
from gpflow.kernels import SquaredExponential, White
from gpflow.optimizers import Scipy
from gpflow.utilities import print_summary, triangular
from gpflow.base import Parameter
from scipy.cluster.vq import kmeans2
from scipy.stats import norm
from scipy.special import logsumexp

from original_datasets import Datasets
from dgp import DGP

output_logdir = '/tmp/tensorboard'

!rm -rf '{output_logdir}'
!mkdir '{output_logdir}'

%load_ext tensorboard

def enumerated_logdir(_logdir_id: int = [0]):
    logdir = Path(output_logdir, str(_logdir_id[0]))
    _logdir_id[0] += 1
    return str(logdir)

In [2]:
datasets = Datasets(data_path='../data/')

In [6]:
data = datasets.all_datasets['concrete'].get_data()
X, Y, Xs, Ys, Y_std = [data[_] for _ in ['X', 'Y', 'Xs', 'Ys', 'Y_std']]

In [18]:
Z = kmeans2(X, 50, minit='points')[0]
D = X.shape[1]

In [26]:
kernels = []
for l in range(3):
    kernels.append(SquaredExponential() + White(variance=2e-6))
    
model = DGP(D, kernels, Gaussian(variance=0.05), Z, 
            num_outputs=Y.shape[1], num_samples=1)

# start inner layers deterministically
for layer in model.layers[:-1]:
    layer.q_sqrt = Parameter(layer.q_sqrt.value() * 1e-5, 
                             transform=triangular())

In [27]:
optimiser = tf.optimizers.Adam(0.01)

def optimisation_step(model):
    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        obj = - model.elbo(X, Y, full_cov=False)
        grad = tape.gradient(obj, model.trainable_variables)
    optimiser.apply_gradients(zip(grad, model.trainable_variables))

In [28]:
def monitored_training_loop(model, logdir,
                            epochs=1, logging_epoch_freq=10):
    summary_writer = tf.summary.create_file_writer(logdir)
    tf_optimisation_step = tf.function(optimisation_step)
#     tf_optimisation_step = optimisation_step
    
    with summary_writer.as_default():
        for epoch in range(epochs):
            
            tf_optimisation_step(model)
                
            epoch_id = epoch + 1
            if epoch_id % logging_epoch_freq == 0:
                tf.print(f'Epoch {epoch_id}: ELBO (train) {model.elbo(X, Y)}')
                
                # computes the mean and variance of the help-out data at 
                # input points
#                 mean, var = model.predict_f_full_cov(Xs, num_samples=10)
                # produces samples from the posterior at input points
#                 samples, _, _ = model.predict_all_layers_full_cov(Xs, num_samples=10)
#                 fig = plotting_regression(X, Y, Xs, mean, var, samples)
                
#                 summary_matplotlib_image(dict(model_samples=fig), step=epoch)
                tf.summary.scalar('elbo', data=model.elbo(X, Y), step=epoch)
                tf.summary.scalar('likelihood/variance', data=model.likelihood.variance, step=epoch)
                tf.summary.scalar('layer1_kernel/lengthscale', data=model.layers[0].kernel.kernels[0].lengthscale, step=epoch)
                tf.summary.scalar('layer1_kernel/variance', data=model.layers[0].kernel.kernels[0].variance, step=epoch)
                tf.summary.scalar('layer2_kernel/lengthscale', data=model.layers[1].kernel.kernels[0].lengthscale, step=epoch)
                tf.summary.scalar('layer2_kernel/variance', data=model.layers[1].kernel.kernels[0].variance, step=epoch)

In [30]:
monitored_training_loop(model, logdir=enumerated_logdir(), 
                        epochs=5000, logging_epoch_freq=200)

Epoch 200: ELBO (train) -1015.1031979066487
Epoch 400: ELBO (train) -833.2478216386254
Epoch 600: ELBO (train) -706.7946142692124
Epoch 800: ELBO (train) -649.4962408972034
Epoch 1000: ELBO (train) -633.1867308808069
Epoch 1200: ELBO (train) -610.3892083505866
Epoch 1400: ELBO (train) -585.285414135895
Epoch 1600: ELBO (train) -608.3614653709841
Epoch 1800: ELBO (train) -574.45752432212
Epoch 2000: ELBO (train) -582.2338566867653
Epoch 2200: ELBO (train) -562.1402554650483
Epoch 2400: ELBO (train) -568.5105722570199
Epoch 2600: ELBO (train) -580.9915916326896
Epoch 2800: ELBO (train) -557.8225633817202
Epoch 3000: ELBO (train) -550.3971204938057
Epoch 3200: ELBO (train) -557.0845845893652
Epoch 3400: ELBO (train) -551.9559737578347
Epoch 3600: ELBO (train) -544.3743208824665
Epoch 3800: ELBO (train) -537.3167615794084
Epoch 4000: ELBO (train) -559.1180490333746
Epoch 4200: ELBO (train) -544.669017419691
Epoch 4400: ELBO (train) -545.7962400503319
Epoch 4600: ELBO (train) -567.489583373

In [31]:
test_ll = model.log_likelihood(Xs, Ys)

In [33]:
print(test_ll / Xs.shape[0])

tf.Tensor(-3.0104994570009054, shape=(), dtype=float64)
